In [1]:
#라이브러리 임포트
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Activation
from keras import regularizers
import numpy as np

#이미지 생성용
from keras.preprocessing.image import ImageDataGenerator
from collections import Counter

# 콜백 함수
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

#시간 구하기용
from datetime import datetime

#디렉토리 생성용
import os

Using TensorFlow backend.


In [2]:
def ModelDesign(size_x, size_y, depth, label_num, regularizer_value):     
    #초기화
    model = Sequential() 
    
    # CNN 레이어
    # 레이어1(Input)
    model.add(Conv2D(filters=96,
                     input_shape=(size_x,size_y,depth),
                     kernel_size=(11,11),
                     strides=(4,4),
                     padding='valid',
                     kernel_regularizer=regularizers.l2(regularizer_value)))
    # Activation
    model.add(Activation('relu'))
    # Pooling 
    model.add(MaxPooling2D((2,2), strides=(1,1), padding='valid'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())
    
    # 레이어2
    model.add(Conv2D(filters=256, kernel_size=(11,11), 
                     strides=(1,1), padding='valid', 
                     kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D((2,2), strides=(1,1), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())
    
    # 레이어3
    model.add(Conv2D(filters=384, kernel_size=(3,3), 
                     strides=(1,1), padding='valid', 
                     kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())
    
    # 레이어4
    model.add(Conv2D(filters=384, kernel_size=(3,3), 
                     strides=(1,1), padding='valid', 
                     kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())
    
    # 레이어5
    model.add(Conv2D(filters=256, kernel_size=(3,3), 
                     strides=(1,1), padding='valid', 
                     kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D((2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())
    
    # Dense 레이어
    # Flatten
    model.add(Flatten())
    
    # 레이어1(Input2)
    model.add(Dense(4096, input_shape=(size_x*size_y*depth,), 
                    kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.8))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 레이어2
    model.add(Dense(4096, kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.8))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 레이어3
    model.add(Dense(1000, kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.8))
    # Batch Normalisation
    model.add(BatchNormalization())
    
    # 출력 레이어
    model.add(Dense(label_num))
    model.add(Activation('softmax'))
    
    #모델 확인
    model.summary()
    
    return model

In [3]:
# 설정값들
IMAGE_SIZE = 128
IMAGE_DEPTH = 3
LABEL_NUM = 16
REGULIZER = 0.0001

#GetCNN Model
model = ModelDesign(IMAGE_SIZE, IMAGE_SIZE, IMAGE_DEPTH, LABEL_NUM, REGULIZER)

#Compling CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 29, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 29, 29, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 19, 256)       2973952   
_________________________________________________________________
activation_2 (Activation)    (None, 19, 19, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 256)       0         
__________

In [4]:
# Val값에 따라 훈련 조기종료 설정
early_stopping = EarlyStopping(patience=2)

# Class Weights 계산
# 가장 많은 클레스의 수 / 각 클레스의 수
# 클레스 = 각 라벨에 해당하는 데이터 수
'''
counter = Counter(training_set.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}
'''
#오늘 날짜_시간
NOW = datetime.now().strftime('%Y%m%d_%H_%M_%S')

#텐서보드 로그
tensorboard = TensorBoard(log_dir="./Logs/AlexNet/" + NOW)

#디렉토리 생성
def MakeDirectory(dir_path):
    try:
        if not(os.path.isdir(dir_path)):
            os.makedirs(os.path.join(dir_path))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise

In [5]:
#훈련 6개의 데이터 셋으로 진행(Validation 데이터 셋이 다름)
DATASET_NUM = 6
EPOCH_NUM = 20
BATCH_SIZE = 32
CYCLE_NUM = 8

#훈련용 이미지 생성기
train_datagen = ImageDataGenerator(horizontal_flip = True)
validation_datagen = ImageDataGenerator(horizontal_flip = True)

# Weight, Model 저장용 폴더 생성
model_save_dir = "./Models/AlexNet/" + NOW
MakeDirectory(model_save_dir)

#모델 저장
model_json = model.to_json()
with open(model_save_dir + "/model_alexnet_" + NOW + ".json", "w") as json_file : 
    json_file.write(model_json)
             
#몇 사이클 돌지
for cycle in range(0, CYCLE_NUM):
    
    print("Cycle ", (cycle + 1))
    
    # 데이터셋 수 만큼 훈련
    for index in range(1, (DATASET_NUM + 1)):

        print("Training ", index)

        # 데이터 셋 생성
        dir_path = "".join(['./Dataset_Multi/Dataset', str(index)])
        training_set = train_datagen.flow_from_directory(dir_path + '/Train',
                                                    target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                    batch_size = BATCH_SIZE)

        validation_set = validation_datagen.flow_from_directory(dir_path + '/Validation',
                                                   target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                   batch_size = BATCH_SIZE)

        #배치 수 계산
        train_step_epoch = int(len(training_set.classes) / BATCH_SIZE)
        validation_step_epoch = int(len(validation_set.classes) / BATCH_SIZE)
        
        #훈련!
        model.fit_generator(training_set,
                            steps_per_epoch = train_step_epoch,
                            epochs = EPOCH_NUM,
                            validation_data = validation_set,
                            validation_steps = validation_step_epoch,
                            #class_weight=class_weights,
                            shuffle=True,
                            callbacks=[tensorboard, early_stopping])

        print("Traing ", index, " done")
        
        #오늘 날짜_시간
        current_time = datetime.now().strftime('%Y%m%d_%H_%M_%S')
        
        # Weight 저장
        model.save_weights(model_save_dir + "/weight_alexnet_" + current_time + ".h5")
        print("Saved model to disk\n")

Cycle  1
Training  1
Found 62573 images belonging to 16 classes.
Found 15875 images belonging to 16 classes.
Epoch 1/20
1955/1955 [==============================] - 186s 95ms/step - loss: 5.1268 - acc: 0.2007 - val_loss: 5.3953 - val_acc: 0.2431
Epoch 2/20
1955/1955 [==============================] - 178s 91ms/step - loss: 5.6883 - acc: 0.2391 - val_loss: 7.0804 - val_acc: 0.2325
Epoch 3/20
1955/1955 [==============================] - 178s 91ms/step - loss: 5.9573 - acc: 0.2495 - val_loss: 6.0716 - val_acc: 0.2532
Traing  1  done
Saved model to disk

Training  2
Found 68806 images belonging to 16 classes.
Found 9642 images belonging to 16 classes.
Epoch 1/20
2150/2150 [==============================] - 709s 330ms/step - loss: 6.1604 - acc: 0.2558 - val_loss: 6.5232 - val_acc: 0.2451
Epoch 2/20
2150/2150 [==============================] - 193s 90ms/step - loss: 6.0247 - acc: 0.2606 - val_loss: 5.7469 - val_acc: 0.2454
Epoch 3/20
2150/2150 [==============================] - 192s 89ms/ste

1946/1946 [==============================] - 178s 91ms/step - loss: 3.4427 - acc: 0.2676 - val_loss: 3.5371 - val_acc: 0.2317
Epoch 5/20
1946/1946 [==============================] - 177s 91ms/step - loss: 3.5158 - acc: 0.2676 - val_loss: 3.5848 - val_acc: 0.2320
Traing  3  done
Saved model to disk

Training  4
Found 64149 images belonging to 16 classes.
Found 14299 images belonging to 16 classes.
Epoch 1/20
2004/2004 [==============================] - 181s 91ms/step - loss: 3.6203 - acc: 0.2544 - val_loss: 3.6064 - val_acc: 0.2862
Epoch 2/20
2004/2004 [==============================] - 182s 91ms/step - loss: 3.6545 - acc: 0.2543 - val_loss: 3.5771 - val_acc: 0.2860
Epoch 3/20
2004/2004 [==============================] - 182s 91ms/step - loss: 3.6642 - acc: 0.2544 - val_loss: 4.2873 - val_acc: 0.2865
Epoch 4/20
2004/2004 [==============================] - 182s 91ms/step - loss: 3.5166 - acc: 0.2544 - val_loss: 3.3888 - val_acc: 0.2863
Epoch 5/20
2004/2004 [==============================

1955/1955 [==============================] - 178s 91ms/step - loss: 3.3025 - acc: 0.2620 - val_loss: 4.3129 - val_acc: 0.2530
Traing  1  done
Saved model to disk

Training  2
Found 68806 images belonging to 16 classes.
Found 9642 images belonging to 16 classes.
Epoch 1/20
2150/2150 [==============================] - 192s 89ms/step - loss: 3.2910 - acc: 0.2623 - val_loss: 3.3255 - val_acc: 0.2452
Epoch 2/20
2150/2150 [==============================] - 192s 89ms/step - loss: 3.3000 - acc: 0.2622 - val_loss: 3.2327 - val_acc: 0.2454
Epoch 3/20
2150/2150 [==============================] - 192s 89ms/step - loss: 3.2864 - acc: 0.2622 - val_loss: 3.2597 - val_acc: 0.2452
Epoch 4/20
2150/2150 [==============================] - 192s 89ms/step - loss: 3.2672 - acc: 0.2625 - val_loss: 3.4239 - val_acc: 0.2442
Traing  2  done
Saved model to disk

Training  3
Found 62272 images belonging to 16 classes.
Found 16176 images belonging to 16 classes.
Epoch 1/20
1946/1946 [==============================]

2139/2139 [==============================] - 191s 89ms/step - loss: 3.1671 - acc: 0.2597 - val_loss: 3.4441 - val_acc: 0.2617
Epoch 6/20
2139/2139 [==============================] - 191s 89ms/step - loss: 3.2175 - acc: 0.2598 - val_loss: 4.6246 - val_acc: 0.2533
Traing  6  done
Saved model to disk

Cycle  6
Training  1
Found 62573 images belonging to 16 classes.
Found 15875 images belonging to 16 classes.
Epoch 1/20
1955/1955 [==============================] - 178s 91ms/step - loss: 3.2337 - acc: 0.2619 - val_loss: 3.1251 - val_acc: 0.2533
Epoch 2/20
1955/1955 [==============================] - 178s 91ms/step - loss: 3.1705 - acc: 0.2620 - val_loss: 3.4707 - val_acc: 0.2536
Epoch 3/20
1955/1955 [==============================] - 178s 91ms/step - loss: 3.1076 - acc: 0.2619 - val_loss: 3.1527 - val_acc: 0.2530
Traing  1  done
Saved model to disk

Training  2
Found 68806 images belonging to 16 classes.
Found 9642 images belonging to 16 classes.
Epoch 1/20
2150/2150 [======================

2062/2062 [==============================] - 185s 90ms/step - loss: 3.1137 - acc: 0.2555 - val_loss: 6.8289 - val_acc: 0.2863
Traing  5  done
Saved model to disk

Training  6
Found 68453 images belonging to 16 classes.
Found 9995 images belonging to 16 classes.
Epoch 1/20
2139/2139 [==============================] - 190s 89ms/step - loss: 3.1163 - acc: 0.2598 - val_loss: 3.8629 - val_acc: 0.2623
Epoch 2/20
2139/2139 [==============================] - 190s 89ms/step - loss: 3.1077 - acc: 0.2598 - val_loss: 3.0857 - val_acc: 0.2622
Epoch 3/20
2139/2139 [==============================] - 190s 89ms/step - loss: 3.0677 - acc: 0.2601 - val_loss: 5.5637 - val_acc: 0.2622
Epoch 4/20
2139/2139 [==============================] - 190s 89ms/step - loss: 3.1187 - acc: 0.2596 - val_loss: 12.2000 - val_acc: 0.2628
Traing  6  done
Saved model to disk

Cycle  8
Training  1
Found 62573 images belonging to 16 classes.
Found 15875 images belonging to 16 classes.
Epoch 1/20
1955/1955 [=====================

In [35]:
#테스트셋 불러오기
test_datagen = ImageDataGenerator(horizontal_flip = True)
test_set = validation_datagen.flow_from_directory('./Dataset/Test',
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(
            test_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")
output = model.predict_generator(test_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_set.class_indices)
print(output[0])

-- Evaluate --
acc: 22.72%
-- Predict --
{'A': 0, 'AB': 1, 'ABC': 2, 'ABCD': 3, 'ABD': 4, 'AC': 5, 'ACD': 6, 'AD': 7, 'B': 8, 'BC': 9, 'BCD': 10, 'BD': 11, 'C': 12, 'CD': 13, 'D': 14, 'N': 15}
[0.001 0.000 0.000 0.000 0.000 0.000 0.000 0.075 0.000 0.047 0.000 0.000
 0.000 0.002 0.002 0.871]


In [ ]:
# 모델 평가하기 - 트레이닝 셋으로
print("-- Evaluate --")
scores = model.evaluate_generator(
            training_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기 - 트레이닝 셋으로
print("-- Predict --")
output = model.predict_generator(training_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(training_set.class_indices)
print(output[0])